In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 17.5MB/s 
     |████████████████████████████████| 3.2MB 34.0MB/s 
     |████████████████████████████████| 890kB 36.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a76ab318f5c6119ee5a402ae1bfec679e2fe032e89c5a6673bf32c201d1de839
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchtext
from torchtext.legacy import data, datasets

import random
import numpy as np
import time
import spacy

import transformers
from transformers import BertTokenizer, BertModel

print(f'torch : {torch.__version__}  |  torchtext : {torchtext.__version__}  |  spacy : {spacy.__version__}  |  np : {np.__version__}  |  transformers : {transformers.__version__}')

torch : 1.8.1+cu101  |  torchtext : 0.9.1  |  spacy : 2.2.4  |  np : 1.19.5  |  transformers : 4.4.2
